# First Steps with Feedback Control

**WARNING:** Some cells in this notebook are not meant to be run sequentially!

In [ ]:
import sys
sys.path.append('..')

In [ ]:
%matplotlib inline  
import numpy as np
import matplotlib.pyplot as plt
import math
import time

In [ ]:
from ctrl.algo import Proportional, PID, VelocityController, OpenLoop

Substitute `HOST` and `PORT` with the IP address and port of your controller:

In [ ]:
from ctrl.client import Controller
HOST, PORT = "192.168.10.106", 9999
controller = Controller(HOST, PORT)

##Static Response of the DC motor

It is helpful to think of the motor as an entity that takes an **input signal**, in this case the **reference** which is translated into a **voltage**, and produces as **output signal** the pulley **velocity**. We represent such **input-output** relationship using a **block-diagram**:

<img src="figure1.png" width="300">

We will now perform some experiments to characterize the behavior of the DC motor. Instead of performing the experiment by hand, we will write python programs to vary the reference input, i.e. voltage, on the motor and then collect and analyze the resulting data.

**TASKS:**
1. Write python code that resets the controller logger, sets the reference at 100% for 2 seconds then at 95% for 2 second and so on until you reach 5%
<br/>
*Hint:* use a `for` loop and `range`; make sure you're using an `OpenLoop` controller
2. Import the data, use `numpy` to calculate the motor velocity and plot the time *vs* the velocity using matplotlib
3. Plot also the reference (3rd column of the data) *vs* the velocity
4. Compare and discuss your results with the other groups

**QUESTIONS:**
1. Is the response of the motor linear?
2. How could we calculate the *gain* of the motor?

In [ ]:
# SOLUTIONS 1,2,3

## Curve fitting

**TASKS:**
1. Use the following code to fit your data to a line of the form y = a x + b
2. Modify the code to fit a line of the form y = a x

**QUESTIONS:**
1. What does the code do?
2. Why is the value of b not zero?

In [ ]:
def func(x, a, b):
    return a * x + b

import scipy.optimize as optimize

pars, residues = optimize.curve_fit(func, reference, velocity)
a, b = pars.tolist()
print('a = {:4.3f}, b = {:4.3f}'.format(a,b))

**TASKS:**
1. Use the values of a and b you calculated and the following code to plot the curve
   $$y = a \, x + b$$
   where $y$ is the velocity and $x$ the reference input along with the data you imported earlier
   
**QUESTIONS:**
1. How well do they match?
2. What is the meaning of the line when reference > 0 but velocity < 0
3. Repeat for a line of the form y = a x 

In [ ]:
plt.plot(reference, velocity, '.', np.array([0,100]), a * np.array([0,100]) + b,'-')
plt.xlabel('reference (%)')
plt.ylabel('velocity (Hz)')
plt.grid()

## Open-loop Control

In open-loop control the **reference signal**, in the case of the DC motor the input **voltage**, is produced by an algorithm, the **controller**, as shown in the following block-diagram:

<img src="figure2.png" width="500">

**TASKS:**
1. Use the line $$y = a x + b$$ where $y$ is the velocity and $x$ is the reference input to calculate the value of $x$ required to make the motor rotate at a constant 8Hz speed
2. Write python code that resets the controller logger, and sets the reference at the value you calculated for 5s
3. Import the data and plot time vs velocity and time vs reference
4. Discard the first second of data and use `np.mean` and `np.std` to calculate the average and standard deviation of the velocity

**QUESTIONS:**
1. How well did your calculation performed?
2. Generalize the calculation to work for any desired velocity: that's an **open-loop controller**

In [ ]:
# SOLUTIONS 4,5,6,7

**TASKS:**
1. Run your code and have your group mate touch the top of the big pulley with his or her finger and apply a **gentle** force trying to stop the motor. Make sure the force **does not completely stop the motor**, as this may damage the motor
2. Repeat the above steps to calculate the mean and standard deviation of the velocity in this case

**QUESTIONS:**
1. Is the average close to 8HZ?
2. Compare the standard deviation of the velocity and the reference for a run in which you were adjusting the potentiometer and a run in which you were not adjusting the potentiometer
3. What happened to the velocity when your group mate touched the pulley?
4. What is causing the change in velocity?

In [ ]:
# YOUR SOLUTION HERE

## You are the Controller

**TASKS:**
1. Adjust the potentiometer in the control box so that it is pointing up
2. Run the following code to setup a display in your notebook

In [ ]:
# Setup display widgets

from IPython.display import display
from IPython.html import widgets

vmax = 12
vmin = -12
N = 200
vbar = widgets.FloatProgress(min=vmin, max=vmax, step = (vmax - vmin)/N)
ptext = widgets.Text('0',width='12ex',border_style='none')
vtext = widgets.Text('0',width='12ex',border_style='none')
row3 = widgets.HBox([widgets.HTML('{:5.3f}&nbsp;'.format(vmin)),
                     vbar,
                     widgets.HTML('&nbsp;{:5.3f}'.format(vmax))])
row2 = widgets.HBox([widgets.HTML('<b>velocity = &nbsp;</b>'),
                     widgets.VBox([vtext, row3])])
row1 = widgets.HBox([widgets.HTML('<b>position = &nbsp;</b>'),
                     ptext])
display(row1,row2)

**TASKS:**
1. Run the following code and look at the velocity reading
2. Adjust the potentiometer so that the velocity reads about 8 Hz
3. Import the data and plot time vs velocity and time vs reference
4. Discard the first 2 seconds of data and use `np.mean` and `np.std` to calculate the average and standard deviation of the velocity

**QUESTIONS:**
1. How well where you able to regulate the velocity? Compare with the open-loop controller
2. Explain in words what you did in order to regulate the voltage at 8Hz
3. Could you regulate velocity without reading the speed, that is without a **sensor**?
4. Explain what you did with the reading of the speed: this is **feedback**

In [ ]:
controller.set_controller1(OpenLoop())
controller.reset_logger()
controller.set_encoder1(0)
Ts = 0.3
K = int(30/Ts)
with controller:
    controller.set_reference1_mode(1)
    old_time, old_position = controller.get_encoder1()
    k = 0
    while k < K:
        current_time, current_position = controller.get_encoder1()
        velocity = (current_position - old_position) / (current_time - old_time)
        vbar.value = -velocity/2
        ptext.value = '{:5.3f}'.format(current_position)
        vtext.value = '{:5.3f}'.format(velocity)
        k = k + 1
        old_time, old_position = current_time, current_position
        time.sleep(Ts)
    controller.set_reference1_mode(0)

In [ ]:
# YOUR SOLUTION HERE

**TASKS:**
1. Repeat the above tasks while a group mate disturbs the wheel

**QUESTIONS:**
1. Is the average close to 8HZ?
2. What happened to the velocity when your group mate touched the pulley? What did you do then to keep the velocity constant?
3. What happened to the velocity when your group mate let go of the pulley? What did you do then to keep the velocity constant?
4. Formulate your strategy as an **algorithm**: that's a **closed-loop controller**

In [ ]:
# YOUR SOLUTION HERE

**TASKS:**
1. Repeat the above steps but this time adjust the potentiometer to try to keep the velocity constant at 8Hz as your group mate disturbs the pulley

**QUESTIONS:**
1. Is the average close to 8HZ?
2. Compare the standard deviation of the velocity and the reference for a run in which you were adjusting the potentiometer and a run in which you were not adjusting the potentiometer
3. Explain in words what you did in order to regulate the voltage at 8Hz
4. Formulate your strategy as an algorithm

In [ ]:
# YOUR SOLUTION HERE

## Closed-loop Control

In closed-loop control a **sensor** is used to measure an output quantity of interest, the velocity in the case of the DC motor, which is compared with a **reference** to produce an **errror signal**. The error is what is used by the **feedback controller** to calculate the input for the system being controlled, in this case the motor input **voltage**. The overal process can be represented by the block-diagram:

<img src="figure3.png" width="500">

The name **feedback** refers to the fact that the system *output* is **fed back** into the system *input* by the *controller*.

One of the simplest feedback controlers is the **proportional controller**:

$$
\begin{aligned}
  \text{control} &= K \, \text{error}, & \text{error} &= \text{reference} - \text{velocity}
\end{aligned}
$$

which produces input which is *proportional* to the error. The constant of proportionallity, $K$, is the controller *gain*.

Because our reference is always a value from 0-100 we need to modify the above diagram:

<img src="figure4.png" width="650">

to allow for different maximum speeds.

**TASKS:**
1. Run the following code to implement a proportional velocity controller
2. Import the data and plot time vs velocity and time vs reference and control
3. Discard the first second of data and use `np.mean` and `np.std` to calculate the average and standard deviation of the velocity
4. Calculate the ratio between the average velocity and the reference velocity

**QUESTIONS:**
1. Did the closed-loop controller achieve the reference velocity? Explain

In [ ]:
Kp = 1
K = Kp / a
vmax = 12
controller.set_controller1(VelocityController(Proportional(K, vmax)))
print('K = {:5.3f}, vmax = {:5.3f}'.format(K,vmax))

In [ ]:
vreference = 8
controller.reset_logger()
controller.set_encoder1(0)
with controller:
    controller.set_reference1(100*vreference/vmax)
    time.sleep(3)
log = controller.get_log()

In [ ]:
# Solutions 8,9

**TASKS:**
1. Repeat the above steps with a proportional gain `Kp` equal to 2 then 4

**QUESTIONS:**
1. How does the ratio between the average velocity and the reference velocity vary with `Kp`?

In [ ]:
# YOUR SOLUTION HERE

### Open- and Closed-loop Gains

In order to understand why the achieved velocity is not equal to the reference velocity it is convinient to think of each block of the control-loop as a simple *static gain*. That is, the motor is simply

$$
    \text{velocity} = G \, \text{control}
$$

where $G$ is the **open-loop gain**, and the controller:

$$
    \text{control} = K \, \text{error}
$$

Recalling that 

$$
\text{error} = \text{reference} - \text{velocity} = \frac{\text{reference}}{100} \, \text{vmax} - \text{velocity}
$$

you will show that:

$$
\begin{aligned}
\frac{\text{velocity}}{\text{vmax}} &= H \, \frac{\text{reference}}{100}, & H &= \frac{K G}{1 + K G}
\end{aligned}
$$

The quantity $H$ is the **closed-loop gain**.

**TASKS:**
1. Use the above formula to compare the ratio between the average velocity and the reference velocity with what you obtained experimentally

**QUESTIONS:**
1. How can you compute the open-loop gain, $G$?
2. Why did we set `K = Kp / a`?
3. What value of `Kp` would make the closed loop gain, $H$, become closed to 1?
4. What is the effect of errors in open-loop gain, $G$, in the value of the closed-loop gain, $H$?
5. Prove the closed-loop gain formula. *Hint: substitute for control and error and solve for velocity in the above equations*

In [ ]:
# YOUR SOLUTION HERE

### Regulation and Instability

**TASKS:**
1. Run the following code to implement a closed-loop proportional velocity controller this time with a reference velocity equal to 0: this is a **regulator**
2. Try to turn the pulley when the controller is on
3. Repeat with `Kp` equal to 2, 4, and 16
4. Repeat with `Kp` equal to -1, -2, and -4

**QUESTION:**
1. What difference does the gain make?
2. Name an application for this velocity regulator
3. What does the regulator do when the gain `Kp` is negative?
4. What happens when the gain `Kp` is positive and large?
5. What happens when the gain `Kp` is negative and large?

In [ ]:
Kp = 1
K = Kp / a
controller.set_controller1(VelocityController(Proportional(K)))
controller.reset_logger()
controller.set_encoder1(0)
with controller:
    controller.set_reference1(0)
    time.sleep(5)
log = controller.get_log()

## Position Control

The principles of feedback control are abstract and apply to a variety of systems. For example, we could use a similar scheme:

<img src="figure5.png" width="650">

if we wish to control the **position** of rotor of the DC motor, instead of its velocity,

In this case **proportional control** is:

$$
\begin{aligned}
  \text{control} &= K \, \text{error}, & \text{error} &= \text{reference} - \text{position}
\end{aligned}
$$

which produces input which is *proportional* to the error, this time computed with respect to position. From a practical standpoint all that is needed is a new sensor to measure position, which we already have.

**TASKS:**
1. Run the following code to implement a proportional position controller
2. Import the data and plot time *vs* position, time *vs* reference and control and time *vs* velocity
3. Repeat the above steps with a proportional gain `Kp` equal to 2 then 4

**QUESTIONS:**
1. Compare the responses in the case of position control with that of velocity control
2. Contrast the control signal in both cases
3. Is position related to the control signal by a gain?
4. How could you calculate the closed-loop gain in the case of position control?

In [ ]:
Kp = 1
K = Kp / a
pmax = 12
controller.set_controller1(Proportional(K, pmax))
print('K = {:5.3f}, pmax = {:5.3f}'.format(K,pmax))

In [ ]:
preference = 10
controller.reset_logger()
controller.set_encoder1(0)
with controller:
    controller.set_reference1(100*preference/pmax)
    time.sleep(3)
log = controller.get_log()

In [ ]:
# Solution 10

#Solutions:

## Solutions 1, 2, 3

In [ ]:
# SOLUTION 1
controller.set_controller1(OpenLoop())
controller.reset_logger()
dT = 2
controller.set_encoder1(0)
with controller:
    time.sleep(1)
    for duty_cycle in range(0,100,5):
        print('\rduty cycle = {}%'.format(100-duty_cycle), end='')
        controller.set_reference1(100-duty_cycle)
        time.sleep(dT)
log = controller.get_log()

In [ ]:
# SOLUTION 2
t = log[:,0]
position = log[:,1]
reference = log[:,2]
velocity = np.zeros(t.shape, float)
velocity[1:] = (position[1:]-position[:-1])/(t[1:]-t[:-1])
plt.plot(t, velocity, '.')
plt.xlabel('t (s)')
plt.ylabel('velocity (Hz)')
plt.grid()

In [ ]:
# SOLUTION 3
plt.plot(reference, velocity, '.')
plt.xlabel('reference (%)')
plt.ylabel('velocity (Hz)')
plt.grid()

## Solutions 4, 5, 6, 7

In [ ]:
# SOLUTION 4
y = 8
x = (y - b)/a
print('x = {}'.format(x))

In [ ]:
# SOLUTION 5
controller.set_controller1(OpenLoop())
controller.reset_logger()
with controller:
    controller.set_reference1(x)
    time.sleep(5)
log = controller.get_log()

In [ ]:
# SOLUTION 6
t = log[:,0]
position = log[:,1]
reference = log[:,2]
velocity = np.zeros(t.shape, float)
velocity[1:] = (position[1:]-position[:-1])/(t[1:]-t[:-1])

plt.subplot(2,1,1)
plt.plot(t, velocity, '.')
plt.xlabel('t (s)')
plt.ylabel('velocity (Hz)')
plt.grid()

plt.subplot(2,1,2)
plt.plot(t, reference, '.')
plt.xlabel('t (s)')
plt.ylabel('reference (%)')
plt.grid()

In [ ]:
# SOLUTION 7
ind, = np.where(t > 1)
vmean = np.mean(velocity[ind[0]:])
vstd = np.std(velocity[ind[0]:])
print('mean = {:5.3f}, standard deviation = {:5.3f}'.format(vmean,vstd))

## Solutions 8, 9

In [ ]:
# SOLUTION 8
t = log[:,0]
position = log[:,1]
reference = log[:,2]
control = log[:,3]
velocity = np.zeros(t.shape, float)
velocity[1:] = (position[1:]-position[:-1])/(t[1:]-t[:-1])

plt.subplot(2,1,1)
plt.plot(t, velocity, '.', t, reference * vmax/100, '-')
plt.xlabel('t (s)')
plt.ylabel('velocity (Hz)')
plt.grid()

plt.subplot(2,1,2)
plt.plot(t, control, '.', t, reference, '-')
plt.xlabel('t (s)')
plt.ylabel('control (%)')
plt.grid()

In [ ]:
# SOLUTION 9
ind, = np.where(t > 1)
vmean = np.mean(velocity[ind[0]:])
vstd = np.std(velocity[ind[0]:])
print('mean = {:5.3f}, standard deviation = {:5.3f}'.format(vmean,vstd))
print('vmean/vreference = {:5.3f}'.format(vmean/vreference))
print('H = {:5.3f}'.format(Kp/(1+Kp)))

## Solution 10

In [ ]:
# SOLUTION 10
t = log[:,0]
position = log[:,1]
reference = log[:,2]
control = log[:,3]
velocity = np.zeros(t.shape, float)
velocity[1:] = (position[1:]-position[:-1])/(t[1:]-t[:-1])

plt.subplot(3,1,1)
plt.plot(t, position, '.')
plt.xlabel('t (s)')
plt.ylabel('position (cycles)')
plt.grid()

plt.subplot(3,1,2)
plt.plot(t, control, '.', t, reference, '-')
plt.xlabel('t (s)')
plt.ylabel('control (%)')
plt.grid()

plt.subplot(3,1,3)
plt.plot(t, velocity, '.')
plt.xlabel('t (s)')
plt.ylabel('velocity (Hz)')
plt.grid()